In [1]:
import pandas as pd # '1.3.5'
import numpy as np # '1.21.5'
import os # VERSION?
from spectral import * # VERSION?
import spectral.io.envi as envi

This notebook describes the harmonization of different spectral libraries (SLIs) for the CarbonMapper Material Fractional Cover product.
The SLIs hamonized in the following are located in the spectra_libraries folder:

* ecospeclib-all
* UCSB
* SLUM
* PHX2022

In [9]:
# get image data to retrieve wavelength settings for spectral resampling of SLIs
img = envi.open("\\\\gdcs-stor.rc.asu.edu\\gdcs-stor\\CarbonMapper\\AlgDevel\\SupplyChain\\MaterialCover\\PHX_June2022\\PHX\\GAO20220716t191531p0000\\GAO20220716t191531p0000_iacorn_refl_ort_sub.hdr")

# Harmonization

SLIs have different spectral band settings and need to be resampled to the sensor's settings.

## ECOSpecLib

In [10]:
# folder where single textfiles were unzipped to
folder = "C:\\Users\\mkoenig3\\Dropbox (ASU)\\Backup\\Documents\\Github\\MaterialFractionalCover\\spectral_libraries\\ecospeclib-all\\"

# Collect meta data from textfiles 
switch = 0 

for f in os.listdir(folder):
    if f.split(".")[-2]=="spectrum":
        f_meta = pd.read_csv(folder+f, nrows=20, sep='^([^:]+):', index_col=0, usecols=[1,2], header=None, encoding='latin-1', engine='python')
        f_meta = f_meta.T
        f_meta["Filename"] = f
        
        if switch==0:
            meta = f_meta.copy()
            switch = 1
        else:
            meta = meta.append(f_meta, ignore_index=True)
                    
# Clean up table
for col in ['Name', 'Type', 'Class', 'Subclass', 'Particle Size']:
    meta[col] = meta[col].str.strip()
    
meta["Type"] = meta["Type"].replace({"rock": "Rock", "Vegetation": "PV", "vegetation": "PV", "manmade": "Manmade", "non photosynthetic vegetation": "NPV", "soil": "Soil"})
meta["Class"] = meta["Class"].replace({"tree": "Tree", "bark": "Bark", "branches": "Branches", "flowers": "Flowers",  "leaves": "Leaves",  "lichen": "Lichen",  "needles": "Needles",  "grass": "Grass"})

# add column "Planet" and reclassify endmembers accordingly
meta.loc[:, "Planet"] = ""
meta.loc[meta.Class=="Concrete", "Planet"] = "Concrete"
meta.loc[meta.Subclass=="Brick", "Planet"] = "Brick"
meta.loc[meta.Subclass=="Cement Cinderblock", "Planet"] = "Concrete"
meta.loc[meta.Subclass=="Cinder", "Planet"] = "Other man-made"
meta.loc[meta.Subclass=="Glas", "Planet"] = "Other man-made"
meta.loc[meta.Subclass=="Marble", "Planet"] = "Other man-made"
meta.loc[meta.Subclass=="Paint", "Planet"] = "Other man-made"
meta.loc[(meta.Class=="General Construction Material") & (meta.Subclass=="Wood"), "Planet"] = "NPV"
meta.loc[meta.Class=="Road", "Planet"] = "Asphalt"
meta.loc[meta.Subclass=="Metal", "Planet"] = "Metal"
meta.loc[meta.Subclass=="Roofing Paper", "Planet"] = "Other man-made"
meta.loc[meta.Name=="Asphalt Shingle", "Planet"] = "Asphalt"
meta.loc[(meta.Class=="Roofing Material") & (meta.Subclass=="Rubber"), "Planet"] = "Other man-made"
meta.loc[meta.Name=="Asphalt roofing shingle", "Planet"] = "Asphalt"
meta.loc[meta.Name=="Reddish Asphalt roofing shingle", "Planet"] = "Asphalt"
meta.loc[meta.Name=="Reddish asphalt Shingle", "Planet"] = "Asphalt"
meta.loc[meta.Name=="Slate stone Shingle", "Planet"] = "Natural substrate"
meta.loc[meta.Name=="Terra cotta Tiles", "Planet"] = "Brick"
meta.loc[meta.Type=="NPV", "Planet"] = "NPV"
meta.loc[(meta.Type=="NPV") & (meta.Class=="Lichen"), "Planet"] = "EXCLUDED"
meta.loc[meta.Type=="PV", "Planet"] = "PV"
meta.loc[meta.Type=="Rock", "Planet"] = "Natural substrate"
meta.loc[meta.Type=="Soil", "Planet"] = "Natural substrate"
meta.loc[meta.Class=="Frost", "Planet"] = "EXCLUDED"
meta.loc[meta.Class=="Ice", "Planet"] = "EXCLUDED"
meta.loc[meta.Class=="Snow", "Planet"] = "EXCLUDED"
meta.loc[meta.Class=="Tap Water", "Planet"] = "Water"
meta.loc[meta.Type=="Meteorites", "Planet"] = "EXCLUDED"
meta.loc[meta.Type=="Mineral", "Planet"] = "EXCLUDED"

# add SLI name: ("sli_name" and "index") will later be the composite key to retrieve endmember-specific information from the original tables
meta["sli_name"] = "ECOSpecLib"

# Filter rows that have values < 450 nm
meta = meta[(meta["First X Value"].astype(float) < 0.450) | (meta["Last X Value"].astype(float) < 0.450)]

# Collect spectral data from textfiles and resample to GAO
switch = 0 

for f in meta["Filename"]: # [meta_vis.Filename=='rock.igneous.felsic.solid.vswir.ward15.jpl.perkin.spectrum.txt']
    
    print(f)

    # read spectrum and wavelengths from file
    spectrum = pd.read_csv(folder+f, skiprows=20, sep='\t', header=None, index_col=0, encoding='latin-1', engine='python')
    spectrum = spectrum.T

    # get wavelengths and values
    wavelengths = spectrum.columns.values.astype(float)
    values = spectrum.iloc[0].values.astype(float)/100
    
    # reverse values and wavelengths in case wavelengths are inverted
    if wavelengths[0] > wavelengths[-1]:
        wavelengths = wavelengths[::-1]
        values = values[::-1]
        
    # resample to GAO
    band_resampler = BandResampler(wavelengths*1000, np.array(img.bands.centers))    
    new_values = band_resampler(values)
    
    # convert to dataframe
    spectrum = pd.DataFrame(new_values, index=np.array(img.bands.centers)).T
    spectrum["Filename"] = f
    
    if switch==0:
        spectra = spectrum.copy()
        switch = 1
    else:
        spectra = spectra.append(spectrum, ignore_index=True)
        
# Merge meta data with spectra
ecospeclib = meta.merge(spectra, left_on="Filename", right_on="Filename")

spectral:INFO: No overlap for target band 0 (347.290009 / 5.009979)
spectral:INFO: No overlap for target band 1 (352.299988 / 5.009995)
spectral:INFO: No overlap for target band 2 (357.309998 / 5.010009)
spectral:INFO: No overlap for target band 3 (362.320007 / 5.005004)
spectral:INFO: No overlap for target band 4 (367.320007 / 5.004990)
spectral:INFO: No overlap for target band 5 (372.329987 / 5.009995)
spectral:INFO: No overlap for target band 6 (377.339996 / 5.010009)
spectral:INFO: No overlap for target band 7 (382.350006 / 5.005005)
spectral:INFO: No overlap for target band 8 (387.350006 / 5.004989)
spectral:INFO: No overlap for target band 9 (392.359985 / 5.009995)
spectral:INFO: No overlap for target band 10 (397.369995 / 5.010010)
spectral:INFO: No overlap for target band 11 (402.380005 / 5.005005)
spectral:INFO: No overlap for target band 12 (407.380005 / 5.005005)
spectral:INFO: No overlap for target band 13 (412.390015 / 5.009995)
spectral:INFO: No overlap for target band 0 

manmade.concrete.constructionconcrete.solid.all.0598uuucnc.jhu.becknic.spectrum.txt
manmade.concrete.pavingconcrete.solid.all.0092uuu_cnc.jhu.becknic.spectrum.txt
manmade.concrete.pavingconcrete.solid.all.0397uuucnc.jhu.becknic.spectrum.txt
manmade.concrete.pavingconcrete.solid.all.0424uuucnc.jhu.becknic.spectrum.txt
manmade.concrete.pavingconcrete.solid.all.0425uuuasp.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.brick.solid.all.0097uuubrk.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.brick.solid.all.0412uuubrk.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.brick.solid.all.0413uuubrk.jhu.becknic.spectrum.txt


spectral:INFO: No overlap for target band 0 (347.290009 / 5.009979)
spectral:INFO: No overlap for target band 1 (352.299988 / 5.009995)
spectral:INFO: No overlap for target band 2 (357.309998 / 5.010009)
spectral:INFO: No overlap for target band 3 (362.320007 / 5.005004)
spectral:INFO: No overlap for target band 4 (367.320007 / 5.004990)
spectral:INFO: No overlap for target band 5 (372.329987 / 5.009995)
spectral:INFO: No overlap for target band 6 (377.339996 / 5.010009)
spectral:INFO: No overlap for target band 7 (382.350006 / 5.005005)
spectral:INFO: No overlap for target band 8 (387.350006 / 5.004989)
spectral:INFO: No overlap for target band 9 (392.359985 / 5.009995)
spectral:INFO: No overlap for target band 10 (397.369995 / 5.010010)
spectral:INFO: No overlap for target band 11 (402.380005 / 5.005005)
spectral:INFO: No overlap for target band 12 (407.380005 / 5.005005)
spectral:INFO: No overlap for target band 13 (412.390015 / 5.009995)
spectral:INFO: No overlap for target band 0 

manmade.generalconstructionmaterial.cementcinderblock.solid.all.0432uuucnc.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.cinder.solid.all.0101uuucin.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.glas.solid.all.0796uuugls.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.marble.solid.all.0722uuumbl.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.paint.solid.all.0385uuupnt.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.paint.solid.all.0386uuupnt.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.paint.solid.all.0402uuupnt.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.paint.solid.all.0403uuupnt.jhu.becknic.spectrum.txt
manmade.generalconstructionmaterial.paint.solid.all.0405uuupnt.jhu.becknic.spectrum.txt


spectral:INFO: No overlap for target band 0 (347.290009 / 5.009979)
spectral:INFO: No overlap for target band 1 (352.299988 / 5.009995)
spectral:INFO: No overlap for target band 2 (357.309998 / 5.010009)
spectral:INFO: No overlap for target band 3 (362.320007 / 5.005004)
spectral:INFO: No overlap for target band 4 (367.320007 / 5.004990)
spectral:INFO: No overlap for target band 5 (372.329987 / 5.009995)
spectral:INFO: No overlap for target band 6 (377.339996 / 5.010009)
spectral:INFO: No overlap for target band 7 (382.350006 / 5.005005)
spectral:INFO: No overlap for target band 8 (387.350006 / 5.004989)
spectral:INFO: No overlap for target band 9 (392.359985 / 5.009995)
spectral:INFO: No overlap for target band 10 (397.369995 / 5.010010)
spectral:INFO: No overlap for target band 11 (402.380005 / 5.005005)
spectral:INFO: No overlap for target band 12 (407.380005 / 5.005005)
spectral:INFO: No overlap for target band 13 (412.390015 / 5.009995)
spectral:INFO: No overlap for target band 0 

...

soil.mollisol.haplustall.none.all.85p4569.jhu.becknic.spectrum.txt
soil.mollisol.paleustoll.none.all.90p186s.jhu.becknic.spectrum.txt
soil.spodosol.cryohumod.none.all.87p4264.jhu.becknic.spectrum.txt
soil.utisol.hapludult.none.all.87p707.jhu.becknic.spectrum.txt


spectral:INFO: No overlap for target band 0 (347.290009 / 5.009979)
spectral:INFO: No overlap for target band 1 (352.299988 / 5.009995)
spectral:INFO: No overlap for target band 2 (357.309998 / 5.010009)
spectral:INFO: No overlap for target band 3 (362.320007 / 5.005004)
spectral:INFO: No overlap for target band 4 (367.320007 / 5.004990)
spectral:INFO: No overlap for target band 5 (372.329987 / 5.009995)
spectral:INFO: No overlap for target band 6 (377.339996 / 5.010009)
spectral:INFO: No overlap for target band 7 (382.350006 / 5.005005)
spectral:INFO: No overlap for target band 8 (387.350006 / 5.004989)
spectral:INFO: No overlap for target band 9 (392.359985 / 5.009995)


soil.vertisol.chromoxerert.none.all.88p475.jhu.becknic.spectrum.txt
vegetation.grass.avena.fatua.vswir.vh352.ucsb.asd.spectrum.txt
vegetation.grass.avena.fatua.vswir.vh353.ucsb.asd.spectrum.txt
vegetation.grass.bromus.diandrus.vswir.vh350.ucsb.asd.spectrum.txt
vegetation.grass.bromus.diandrus.vswir.vh351.ucsb.asd.spectrum.txt
vegetation.grass.unknown.unknown.all.grass.jhu.becknic.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh033.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh034.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh041.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh042.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh043.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh044.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh111.ucsb.asd.spectrum.txt
vegetation.shrub.adenostoma.fasciculatum.vswir.vh112.ucsb.asd.spectrum.txt
vege

spectral:INFO: No overlap for target band 0 (347.290009 / 5.009979)
spectral:INFO: No overlap for target band 1 (352.299988 / 5.009995)
spectral:INFO: No overlap for target band 2 (357.309998 / 5.010009)
spectral:INFO: No overlap for target band 3 (362.320007 / 5.005004)
spectral:INFO: No overlap for target band 4 (367.320007 / 5.004990)
spectral:INFO: No overlap for target band 5 (372.329987 / 5.009995)
spectral:INFO: No overlap for target band 6 (377.339996 / 5.010009)
spectral:INFO: No overlap for target band 7 (382.350006 / 5.005005)
spectral:INFO: No overlap for target band 8 (387.350006 / 5.004989)
spectral:INFO: No overlap for target band 9 (392.359985 / 5.009995)


water.snow.finegranular.fine.all.fine_snw_.jhu.becknic.spectrum.txt
water.snow.mediumgranular.medium.all.medgran_snw_.jhu.becknic.spectrum.txt
water.tapwater.none.liquid.all.tapwater.jhu.becknic.spectrum.txt


## UCSB

In [13]:
ucsb = pd.read_csv("C:\\Users\\mkoenig3\\Dropbox (ASU)\\Backup\\Documents\\Github\\MaterialFractionalCover\\spectral_libraries\\UCSB\\urban-reflectance-spectra-from-santa-barbara--ca.csv")

meta = ucsb[ucsb.columns[:-1075]]
spectra = ucsb[ucsb.columns[-1075:]]/10000

Planet = {'ANNUAL_FORB': 'PV', # herb
          'ASPHALT_GRAVEL': 'Asphalt',
          'ASPHALT_PARKING_LOT': 'Asphalt',
          'ASPHALT_ROAD': 'Asphalt', 
          'ASPHALT_ROOF': 'Asphalt', 
          'BARK': 'NPV', 
          'BOUGAINVILLEA': 'PV',
          'BRICK': 'Brick',
          'BRICK_SIDEWALK': 'Brick', 
          'COMP_SHINGLE_ROOF': 'Other man-made', 
          'CONCRETE_BRIDGE': 'Concrete',
          'CONCRETE_GRAVEL': 'Concrete', 
          'CONCRETE_PARKING_LOT': 'Concrete',
          'CONCRETE_ROAD': 'Concrete', 
          'CONCRETE_SIDEWALK': 'Concrete',
          'CONCRETE_TILE_ROOF': 'Concrete',
          'ENGLISH_IVY': 'PV', 
          'GLASS_ROOF': 'Other man-made', 
          'GRAVEL': 'Natural substrate',
          'GRAVEL_ROOF': 'Natural substrate', 
          'IRGR': 'PV', # grass
          'LICHEN': 'EXCLUDED', 
          'METAL_MANHOLE': 'Metal',
          'METAL_ROOF': 'Metal', 
          'MIRRA': 'PV',
          'NPV': 'NPV', 
          'PAINT': 'Other man-made', 
          'PAINTED_ROOF': 'Other man-made', 
          'PALM': 'PV', 
          'SAND': 'Natural substrate', 
          'SOIL': 'Natural substrate', 
          'STAR_JASMINE': 'PV', 
          'TILE_ROAD': 'Other man-made',
          'TILE_ROOF': 'Other man-made', 
          'WOOD': 'NPV',
          'WOOD_SHINGLE_ROOF': 'NPV'
          }

# add column "Planet" and reclassify endmembers accordingly
meta.loc[:, "Planet"] = meta["Level_7"].replace(Planet)

# add SLI name: ("sli_name" and "index") will later be the composite key to retrieve endmember-specific information from the original tables
meta.loc[:, "sli_name"] = "UCSB"

# Get wavelengths and values
wavelengths = spectra.columns.values.astype(float)
values = spectra.values

# resample to GAO
band_resampler = BandResampler(centers1 = wavelengths, 
                               centers2 = np.array(img.bands.centers), 
                               fwhm1 = None, 
                               fwhm2 = img.bands.bandwidths)     
new_values = band_resampler(values.T)

spectra = pd.DataFrame(new_values, index = np.array(img.bands.centers)).T

ucsb_gao = meta.merge(spectra, how='left', left_index=True, right_index=True)

C:\Users\mkoenig3\Anaconda3\envs\CarbonMapper_py3713\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


## SLUM

In [15]:
# read meta data
meta = pd.read_csv("C:\\Users\\mkoenig3\\Dropbox (ASU)\\Backup\\Documents\\Github\\MaterialFractionalCover\\spectral_libraries\\SLUM\\LUMA_SLUM_META.csv", index_col=0)

# add column "Planet" and reclassify endmembers accordingly
meta.loc[:, "Planet"] = ""
meta.loc[meta.Superclass=="Asphalt", "Planet"] = "Asphalt"
meta.loc[(meta.Superclass=="Brick") & (meta.Class=="Cement brick"), "Planet"] = "Concrete"
meta.loc[(meta.Superclass=="Brick") & (meta.Class=="Ceramic brick"), "Planet"] = "Brick"
meta.loc[(meta.Superclass=="Brick") & (meta.Material=="Ceramic brick with cement"), "Planet"] = "Other man-made"
meta.loc[(meta.Superclass=="Brick") & (meta.Material=="Ceramic brick with paint"), "Planet"] = "Other man-made"
meta.loc[(meta.Superclass=="Brick") & (meta.Material=="Ceramic with cement"), "Planet"] = "Other man-made"
meta.loc[(meta.Superclass=="Brick") & (meta.Material=="Ceramic with paint"), "Planet"] = "Other man-made"
meta.loc[meta.Superclass=="Concrete/Cement", "Planet"] = "Concrete"
meta.loc[meta.Class=="Granite", "Planet"] = "Natural substrate"
meta.loc[(meta.Class=="Granite") & (meta.Material=="Granite with cement"), "Planet"] = "Other man-made"
meta.loc[meta.Class=="Metal", "Planet"] = "Metal"
meta.loc[(meta.Class=="Metal") & (meta.Material=="Metal with paint"), "Planet"] = "Metal"
meta.loc[meta.Superclass=="PVC", "Planet"] = "Plastic"
meta.loc[meta.Superclass=="Quartzite", "Planet"] = "Natural substrate"
meta.loc[meta.Superclass=="Shingle", "Planet"] = "Natural substrate"
meta.loc[(meta.Superclass=="Shingle") & (meta.Material=="Fiber cement"), "Planet"] = "Concrete"
meta.loc[meta.Superclass=="Stone", "Planet"] = "Natural substrate"
meta.loc[(meta.Superclass=="Tile") & (meta.Material=="Cement"), "Planet"] = "Concrete"
meta.loc[(meta.Superclass=="Tile") & (meta.Material=="Ceramic"), "Planet"] = "Brick"

# add SLI name: ("sli_name" and "index") will later be the composite key to retrieve endmember-specific information from the original tables
meta.loc[:, "sli_name"] = "SLUM"

# read spectra
spectra = pd.read_csv("C:\\Users\\mkoenig3\\Dropbox (ASU)\\Backup\\Documents\\Github\\MaterialFractionalCover\\spectral_libraries\\SLUM\\LUMA_SLUM_SW.csv", index_col=0).T

# get wavelengths, values, and ID 
wavelengths = spectra.columns.values.astype(float)
values = spectra.values
idx = spectra.index

# resample to GAO
band_resampler = BandResampler(centers1 = wavelengths, 
                               centers2 = np.array(img.bands.centers), 
                               fwhm1 = None, 
                               fwhm2 = img.bands.bandwidths)                                
new_values = band_resampler(values.T)
spectra = (pd.DataFrame(new_values, index = np.array(img.bands.centers)).T).set_index(idx)/100

# merge meta with spectra and write to file
slum_gao = meta.merge(spectra, how='left', left_index=True, right_index=True).reset_index()

## PHX

### GAO20220716t191531p0000_iacorn_refl_ort

In [17]:
phx_gao = pd.read_csv("C:\\Users\\mkoenig3\\Dropbox (ASU)\\Backup\\Documents\\Github\\MaterialFractionalCover\\spectral_libraries\\PHX2022\\GAO20220716t191531p0000_iacorn_refl_ort_sli.csv")

# clean up DataFrame
phx_gao['name'] = phx_gao['name'].apply(lambda x: x[:-2])
phx_gao = phx_gao.drop(columns=['source'])
phx_gao.insert(1, "sli_name", "GAO20220716t191531p0000_iacorn_refl_ort")

# add column "Planet" and reclassify endmembers accordingly
phx_gao.loc[:, "Planet"] = ""
phx_gao.loc[phx_gao.name=="pool1", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool2", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool3", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool4", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool5", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool6", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool7", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool8", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool9", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool10", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool11", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool12", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool13", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool14", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool15", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="pool16", "Planet"] = "Water"
phx_gao.loc[phx_gao.name=="blue_tarp", "Planet"] = "Plastic"
phx_gao.loc[phx_gao.name=="yellow_tarp", "Planet"] = "Plastic"
phx_gao.loc[phx_gao.name=="light_blue_tarp", "Planet"] = "Plastic"
phx_gao.loc[phx_gao.name=="reddish_gravel", "Planet"] = "Natural substrate"
phx_gao.loc[phx_gao.name=="playground_sand", "Planet"] = "Natural substrate"
phx_gao.loc[phx_gao.name=="red_shipping_container", "Planet"] = "Metal"
phx_gao.loc[phx_gao.name=="grey_bare_metal_roof", "Planet"] = "Metal"
phx_gao.loc[phx_gao.name=="white_flat_roof", "Planet"] = "Other man-made"
phx_gao.loc[phx_gao.name=="yellow_painted_metal_roof", "Planet"] = "Metal"
phx_gao.loc[phx_gao.name=="terracotta_shingle", "Planet"] = "Brick"
phx_gao.loc[phx_gao.name=="grey_bare_metal_roof", "Planet"] = "Metal"
phx_gao.loc[phx_gao.name=="metal_roof", "Planet"] = "Metal"
phx_gao.loc[phx_gao.name=="red_brown_metal_roof", "Planet"] = "Metal"
phx_gao.loc[phx_gao.name=="baseball_infield_clay", "Planet"] = "Natural substrate"
phx_gao.loc[phx_gao.name=="concrete1", "Planet"] = "Concrete"
phx_gao.loc[phx_gao.name=="concrete2", "Planet"] = "Concrete"
phx_gao.loc[phx_gao.name=="asphalt1", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt2", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt3", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt4", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt5", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt6", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt7", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt8", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt9", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt10", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt11", "Planet"] = "Asphalt"
phx_gao.loc[phx_gao.name=="asphalt12", "Planet"] = "Asphalt"

# take mean spectrum as endmember
phx_gao_mean = (phx_gao.groupby(["name","Planet","sli_name"]).mean()/10000).reset_index()

# rename cols
phx_gao_mean.columns = np.append(phx_gao_mean.columns[:3].values, np.array(img.bands.centers))

# Combination & Preparation

In [19]:
# reset_index results in a new col "level_0" that has the original indices from the original tables
combined_sli = pd.concat([slum_gao, ucsb_gao, ecospeclib, phx_gao_mean]).reset_index()
combined_sli = combined_sli.rename(columns={"index": "original_index"})

# get rid of rows with only NaN reflectance
combined_sli = combined_sli[~combined_sli[np.array(img.bands.centers)].isnull().all(axis=1)]

# get rid of Minerals and EXCLUDED EMs
combined_sli = combined_sli[(combined_sli.Type != "Mineral") & (combined_sli.Planet != "EXCLUDED")]

# # get rid of bands that have any NaNs
# combined_sli = combined_sli[combined_sli.columns[~np.isin(combined_sli.columns.values, combined_sli.columns[10:428+10][combined_sli[combined_sli.columns[10:428+10]].isna().any(axis=0)].values)]]

# sort cols so that spectra are the last 428 cols
combined_sli = combined_sli[combined_sli.columns[:10].to_list() + combined_sli.columns[10+428:].to_list() + combined_sli.columns[10:10+428].to_list()]

# replace zeros with 10**-6
combined_sli

# combined_sli.to_csv("C:\\Users\\mkoenig3\\Dropbox (ASU)\\Backup\\Documents\\Github\\MaterialFractionalCover\\spectral_libraries\\CarbonMapper_sli.csv")